In [1]:
import pandas as pd
import json

In [ ]:
food_codes = pd.read_csv("data/food_codes_v3.csv", index_col=0)
food_codes = food_codes.reset_index().set_index("fdc_id")
food_nutrient_amounts = pd.read_csv("data/food_nutrient_amounts.csv", index_col=0)
food_codes["wheat_free"] = food_codes.wheat ^ 1
food_codes["nut_free"] = food_codes.nuts ^ 1
food_codes["fish_shellfish_free"] = food_codes.fish_shellfish ^ 1
food_codes["egg_free"] = food_codes.eggs ^ 1
food_codes["soy_free"] = food_codes.soy ^ 1
food_codes = food_codes.drop(columns=["wheat", "nuts", "fish_shellfish", "eggs", "soy"])

In [72]:
food_nutrient_amount_dict = (
    food_nutrient_amounts.set_index("fdc_id").iloc[:, 2:].T.to_dict()
)
food_dict = food_codes[
    ["food_name", "price", "lbs", "g", "price_per_100_g"]
].T.to_dict()
food_restrictions = [
    r
    for r in food_codes.columns
    if not r in ["food_name", "fdc_id_fndds", "price", "lbs", "g", "price_per_100_g"]
]
restriction_dict = (
    food_codes[food_restrictions].T.map(lambda x: True if x == 1 else False).to_dict()
)
no_nutrition = []
for fdc_id in food_dict:
    food_dict[fdc_id]["restrictions"] = restriction_dict[fdc_id]
    if fdc_id not in food_nutrient_amount_dict:
        no_nutrition.append(fdc_id)
    food_dict[fdc_id]["food_nutrition"] = food_nutrient_amount_dict.get(fdc_id)

for fdc_id in no_nutrition:
    del food_dict[fdc_id]

with open("data/food_data.json", "w") as f:
    json.dump(food_dict, f)

In [73]:
for fdc_id, food in food_dict.items():
    if "food_nutrition" not in food or food["food_nutrition"] is None:
        print(food["food_name"])